In [10]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import torch

from rdkit import Chem
from transformers import AutoModelForMaskedLM, AutoTokenizer, pipeline, RobertaModel, RobertaTokenizer
from typing import List

# import MoleculeNet loaders from DeepChem
from deepchem.molnet import load_bbbp, load_clearance, load_clintox, load_delaney, load_hiv, load_qm7, load_tox21


In [11]:
import deepchem as dc
print(dir(dc.molnet))

# Load the Freesolv dataset
#tasks, datasets, transformers = dc.molnet.load_freesolv(featurizer="ECFP", splitter="random")


['TransformerGenerator', '_MolnetLoader', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'dnasim', 'featurizers', 'load_Platinum_Adsorption', 'load_bace_classification', 'load_bace_regression', 'load_bandgap', 'load_bbbc001', 'load_bbbc002', 'load_bbbp', 'load_cell_counting', 'load_chembl', 'load_chembl25', 'load_clearance', 'load_clintox', 'load_delaney', 'load_factors', 'load_function', 'load_hiv', 'load_hopv', 'load_hppb', 'load_kaggle', 'load_kinase', 'load_lipo', 'load_mp_formation_energy', 'load_mp_metallicity', 'load_muv', 'load_nci', 'load_pcba', 'load_pdbbind', 'load_perovskite', 'load_ppb', 'load_qm7', 'load_qm8', 'load_qm9', 'load_sampl', 'load_sider', 'load_sweet', 'load_thermosol', 'load_tox21', 'load_toxcast', 'load_uspto', 'load_uv', 'load_zinc15', 'motif_density', 'simple_motif_embedding', 'simulate_motif_counting', 'simulate_motif_density_localization', 'simulate_single_motif_detection', 'splitters'

In [22]:
import rdkit
from torch_geometric.datasets import MoleculeNet
import pandas as pd

# Load the Free Energy of Solvation dataset
dataset = MoleculeNet(root=".", name="FreeSolv")

data = pd.DataFrame(columns = ['smiles', 'solvation'])

for ii in range(len(dataset)):
    smiles_name = dataset[ii]["smiles"]
    solv_val = dataset[ii].y.tolist()[0][0]
    # print(smiles_name)
    # print(solv_val)
    data = data.append({'smiles': smiles_name, 'solvation':solv_val}, ignore_index=True)

print(data)

                     smiles  solvation
0    CN(C)C(=O)c1ccc(cc1)OC     -11.01
1              CS(=O)(=O)Cl      -4.87
2                  CC(C)C=C       1.83
3                CCc1cnccn1      -5.45
4                  CCCCCCCO      -4.21
..                      ...        ...
637          CCCCCCCC(=O)OC      -2.04
638                 C1CCNC1      -5.48
639          c1cc(ccc1C=O)O      -8.83
640               CCCCCCCCl       0.29
641                C1COCCO1      -5.06

[642 rows x 2 columns]


In [3]:
import torch
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from transformers import AutoTokenizer, AutoModel

# Load the tokenizers and models
tokenizer = AutoTokenizer.from_pretrained('DeepChem/ChemBERTa-77M-MLM')
model_before = AutoModel.from_pretrained('DeepChem/ChemBERTa-77M-MLM')

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

C:\Users\NajibS\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\NajibS\.cache\huggingface\hub\models--DeepChem--ChemBERTa-77M-MLM. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/6.96k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.26k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/420 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/13.7M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MLM and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [80]:
def calculate_embeddings(df, model, tokenizer):
    text_data = df['smiles'].tolist()
    labels = df['solvation'].tolist()
    inputs = tokenizer(text_data, padding=True, truncation=True, return_tensors='pt', max_length=128)

    with torch.no_grad():
        outputs = model(**inputs)
        
        # Get the embeddings for all tokens, then average them
        embeddings = outputs.last_hidden_state  # Shape: (batch_size, sequence_length, hidden_size)
        embeddings = embeddings.mean(dim=1) 
        embeddings = embeddings.cpu().numpy()
        
    return embeddings, labels


mol_embedding, labels = calculate_embeddings(data, model_before, tokenizer)

In [81]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split



batch_size = 32
test_size = 0.1



x = torch.tensor(mol_embedding, dtype = torch.float32)
y = torch.tensor(labels, dtype = torch.float32).view(-1,1)

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=test_size)


train_dataset = TensorDataset(x_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last = True)


test_dataset = TensorDataset(x_test, y_test)
test_dataloader = DataLoader(test_dataset,batch_size=batch_size, shuffle=True, drop_last=True)


In [82]:
class SolvationDecoder(nn.Module):
    def __init__(self, input_size):
        super(SolvationDecoder, self).__init__()
        self.fc = nn.Linear(input_size,1, bias=True)

    def forward(self, x):
        return self.fc(x)

model = SolvationDecoder(input_size=x_train.shape[1])


In [93]:
epochs = 5000
learning_rate = 1e-4

pretrained = True

model_name = "ChemBerta.pth"
if pretrained:
    model.load_state_dict(torch.load(model_name))


criterion = torch.nn.MSELoss()
optimiser = optim.Adam(model.parameters(), lr = learning_rate)

for epoch in range(epochs):
    model.train()
    train_loss = 0
    test_loss = 0
    for x, y in train_dataloader:
        optimiser.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        loss.backward()
        optimiser.step()
        train_loss += loss.item()

    train_loss /= len(train_dataloader)

    model.eval()
    with torch.no_grad():
        for x, y in test_dataloader:
            y_pred = model(x)
            loss = criterion(y_pred, y)
            test_loss += loss.item()

        test_loss /= len(test_dataloader)

    if epoch % 100 == 0:
        print(f"Epoch: {epoch}, training_loss: {train_loss}, test_loss: {test_loss}")

    

Epoch: 0, training_loss: 1.5169577697912853, test_loss: 2.888656973838806
Epoch: 100, training_loss: 1.50727383295695, test_loss: 2.9832524061203003
Epoch: 200, training_loss: 1.4966876341236963, test_loss: 2.7927417755126953
Epoch: 300, training_loss: 1.4890530738565657, test_loss: 2.9662729501724243
Epoch: 400, training_loss: 1.4752937621540494, test_loss: 2.8949761390686035
Epoch: 500, training_loss: 1.4680695898003049, test_loss: 2.96557879447937
Epoch: 600, training_loss: 1.4617507888211145, test_loss: 2.918785572052002
Epoch: 700, training_loss: 1.4545066588454776, test_loss: 2.556573271751404
Epoch: 800, training_loss: 1.445555845896403, test_loss: 2.959001064300537
Epoch: 900, training_loss: 1.4385487602816687, test_loss: 2.8915200233459473
Epoch: 1000, training_loss: 1.4299376573827531, test_loss: 2.936295509338379
Epoch: 1100, training_loss: 1.4232320388158162, test_loss: 2.957377076148987
Epoch: 1200, training_loss: 1.4152047071192, test_loss: 2.9660156965255737
Epoch: 1300,

In [94]:
torch.save(model.state_dict(), model_name)

In [97]:
import pandas as pd 

# Analyze the results for one batch
y_real = []
y_pred = []
with torch.no_grad():
    model.eval()
    for x,y in train_dataloader:
        pred = model(x)
        y_real.append(y.tolist())
        y_pred.append(pred.tolist())

    y_pred = torch.reshape(torch.tensor(y_pred), (-1,1))
    y_real = torch.reshape(torch.tensor(y_real), (-1,1))

    y_pred = torch.squeeze(y_pred)
    y_real = torch.squeeze(y_real)


In [98]:
import plotly.graph_objects as go
from plotly.offline import iplot

trace = go.Scatter(x = y_real,y = y_pred, mode = "markers")
line_trace = go.Scatter(x = [min(y_real), max(y_real)],
                        y = [min(y_real), max(y_real)],
                        mode = "lines",
                        marker=dict(color='blue'),
                        name='Perfect Prediction')
layout = go.Layout(title = "predicted values vs real values", xaxis=dict(title = "real"),
                   yaxis = dict(title = "pred"), width = 800, height = 600)

figure = go.Figure(data = [trace, line_trace], layout=layout)

iplot(figure)